In [ ]:
#trial creating ngram tree


In [1]:
library(dplyr)
library(tidytext)
library(tidyverse)
library(data.tree)
library(stringr)



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ readr   1.3.1
✔ tibble  2.1.1     ✔ purrr   0.3.2
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ ggplot2 3.1.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [ ]:
subsampledText <- readRDS(file='../data/subsets/enUsTwitter10kSubset.rds')


In [2]:
filepath = '..//data//final//en_US//en_US.twitter.txt'
con = file(filepath, "r")
subsampledText <- readLines(con)

Warning message in readLines(con):
“line 167155 appears to contain an embedded nul”Warning message in readLines(con):
“line 268547 appears to contain an embedded nul”Warning message in readLines(con):
“line 1274086 appears to contain an embedded nul”Warning message in readLines(con):
“line 1759032 appears to contain an embedded nul”

In [78]:
head(subsampledText)

[1] "How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long."  
[2] "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason."
[3] "they've decided its more fun if I don't."                                                                       
[4] "So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)"                           
[5] "Words from a complete stranger! Made my birthday even better :)"                                                
[6] "First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!"

In [79]:
#copy/paste processing from
# https://www.tidytextmining.com/tidytext.html

sosToken <- 'start-of-sequence' #'root' token, representing start of a sequence
eolToken <- 'eol' #token to capture end of line

text <- unlist(subsampledText)

text_df <- tibble(line = 1:length(text), text = text)
text_df <- mutate(text_df, text = paste(text, eolToken))

words <- text_df %>%
  unnest_tokens(word, text) %>%
  select(word)

words$index <- 1:nrow(words)

head(text_df)

head(words, 20)
# tail(words)

line,text
<int>,<chr>
1,"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long. eol"
2,When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason. eol
3,they've decided its more fun if I don't. eol
4,So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;) eol
5,Words from a complete stranger! Made my birthday even better :) eol
6,First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go! eol


word,index
<chr>,<int>
how,1
are,2
you,3
btw,4
thanks,5
for,6
the,7
rt,8
you,9


In [ ]:
#simple test problem
words <- tibble(
    word = c('a', 'b', 'c', 'a', 'b', 'c', 'a', 'c', 'a', 'c', eolToken),
    index = 1:11
)

In [171]:
expandNode <- function(node) {
#     print("expanding")
#     print(node)
    
    #table to record newly created nodes from the expansion:
#     newNodes <- tibble(nodeAbsPath=character(), nOccurrences=integer())
    
    newNodes <- list()
    
#     if (node$word == eolToken){
# #         print("EOL token: will not expand")
#         return(newNodes)
#     }
    
    nextWordIndices <- getOccurrences(node) + 1
    nextWords <- words[nextWordIndices,]
    
    parentPath <- node$absPath
    parentSequence <- node$sequence
#     print(node, 'sequence')
    wordCounts <- nextWords %>%
        count(word) %>%
        rename(nOccurrences=n) %>%
        arrange(desc(nOccurrences))
    
    wordCounts <- filter(wordCounts, nOccurrences > 500)
    wordCounts <- filter(wordCounts, word != eolToken)

    #create dataframe representing new nodes
#     print(parentSequence)
    print(nrow(wordCounts))
    if (nrow(wordCounts) > 0)
    {
        newNodes <- as.Node(
            wordCounts %>%
                mutate(name=as.character(row_number() + nodeCount)) %>% 
                mutate(absPath = paste(parentPath, name, sep='/')) %>%
                mutate(pathString = paste0('./', name)) %>% 
                mutate(sequence = paste(parentSequence, word, sep=' '))
        )$children

        for (newNode in newNodes){ 
            node$AddChildNode(newNode)
        }
    }
    else
    {
        newNodes = list()
    }
    

#     for (uniqueWord in unique(nextWords$word)){
        
#         newNode <- node$AddChild(nodeCount)

#         nodeCount <- nodeCount + 1
# #         newNode$occurrences <- filter(nextWords, word==uniqueWord)$index
#         occurrences <- filter(nextWords, word==uniqueWord)$index
#         newNode$nOccurrences <- length(occurrences)
#         newNode$word <- uniqueWord
#         nodeAbsPath <- newNode$pathString
#         nodeAbsPath <- str_replace(nodeAbsPath, '1', '.') # abs to relative path
#         newNodes <- bind_rows(
#             tibble(nodeAbsPath=nodeAbsPath, nOccurrences=newNode$nOccurrences),
#             newNodes
#         )
#     }
#     if (nrow(newNodes) > 0)
#     {
#         newNodes <- arrange(newNodes, desc(nOccurrences))
#         Sort(node, 'nOccurrences', decreasing = T)
#     }
#     ToDataFrameTable(newNodes, 'nOccurrences', 'absPath')
    newNodes
#     tibble(node = newNodes, nOccurrences = sapply(newNodes, function(n) n$nOccurrences))
#     newNodes <- tibble(node = c(rootNode), nOccurrences=rootNode$nOccurrences)
}

In [172]:
getOccurrences <- function(node) {
    #return a list of indices indicating where the sequence represented by given node occurs in the corpus
    if (!is.null(node$occurrences)){
        #if already stored in the node, just return the value
        occurrences <- node$occurrences
    }
    else {
        #otherwise, lookup based on parent occurences
        parentOccurences <- getOccurrences(node$parent)
        nextWords <- words[parentOccurences+1,]
        occurrences <- filter(nextWords, word==node$word)$index
        node$occurrences <- occurrences
    }
    occurrences
}

In [173]:
expandFrontier <- function(frontier, rootNode) {
    #expand next node:
    # take the next most common sequence in the frontier                      
    nodeToExpand <- frontier$node[[1]]

    # remove it from the frontier
    frontier <- frontier[-1,]

    # expand it
    newNodes <- expandNode(nodeToExpand)
    #     print("new nodes")
    #     print(newNodes)

    # add new nodes to frontier
    #     frontier <- bind_rows(frontier, newNodes)
    
    if (length(newNodes) > 0) {
        #dont add nodes below a certain occurrence threshold to the frontier
        newNodes <- newNodes[sapply(newNodes, function(n) n$nOccurrences > 1)]

        frontier <- bind_rows(frontier, tibble(node = newNodes, nOccurrences = sapply(newNodes, function(n) n$nOccurrences)))
        frontier <- arrange(frontier, desc(nOccurrences))        
    }
    

#     ToDataFrameTable(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')

    frontier
}    


In [174]:
makeRootNode <- function(){
    rootNode <- Node$new(nodeCount)
    rootNode$word <- sosToken
    rootNode$occurrences <- 0:(nrow(words)-1)
    rootNode$nOccurrences <- length(rootNode$occurrences)
    rootNode$absPath <- '.'
    rootNode$sequence <- ''
    rootNode
}

In [175]:
nodeCount <- 1
rootNode <- makeRootNode()
# frontier <-tibble(absPath='.', nOccurrences=rootNode$nOccurrences)
frontier <- tibble(node = c(rootNode), nOccurrences=rootNode$nOccurrences)
frontier$node[[1]]

iterationCount <- 0
expansionHistory <- tibble(iteration=0, elapsedSeconds=0, frontierSize=1)

  levelName
1         1

In [176]:
profilePath <- Rprof("/tmp/rprof-output")
startTime <- Sys.time()
minutesToRun = 1200



while ((nrow(frontier) > 0) && ( as.double(Sys.time() - startTime, units="mins") < minutesToRun)){
    print('Expand frontier')
    print(nrow(frontier))
    print("Elapsed time:")
    print(Sys.time() - startTime)
    flush.console()
    frontier <- expandFrontier(frontier, rootNode)
    
    iterationCount <- iterationCount + 1
    expansionHistory <- bind_rows(expansionHistory, tibble(iteration=iterationCount, elapsedSeconds= as.double(Sys.time() - startTime, units="secs"), frontierSize=nrow(frontier)))
    
}
Rprof(NULL)
expansionHistory

[1] "Expand frontier"
[1] 1
[1] "Elapsed time:"
Time difference of 0.002504349 secs
[1] 4342
[1] "Expand frontier"
[1] 4342
[1] "Elapsed time:"
Time difference of 13.68201 secs
[1] 276
[1] "Expand frontier"
[1] 4617
[1] "Elapsed time:"
Time difference of 16.77192 secs
[1] 183
[1] "Expand frontier"
[1] 4799
[1] "Elapsed time:"
Time difference of 21.38763 secs
[1] 148
[1] "Expand frontier"
[1] 4946
[1] "Elapsed time:"
Time difference of 23.92479 secs
[1] 164
[1] "Expand frontier"
[1] 5109
[1] "Elapsed time:"
Time difference of 27.60731 secs
[1] 156
[1] "Expand frontier"
[1] 5264
[1] "Elapsed time:"
Time difference of 32.22902 secs
[1] 111
[1] "Expand frontier"
[1] 5374
[1] "Elapsed time:"
Time difference of 34.81835 secs
[1] 72
[1] "Expand frontier"
[1] 5445
[1] "Elapsed time:"
Time difference of 39.17139 secs
[1] 87
[1] "Expand frontier"
[1] 5531
[1] "Elapsed time:"
Time difference of 41.3014 secs
[1] 51
[1] "Expand frontier"
[1] 5581
[1] "Elapsed time:"
Time difference of 43.82679 secs

[1] 8
[1] "Expand frontier"
[1] 7662
[1] "Elapsed time:"
Time difference of 3.9806 mins
[1] 14
[1] "Expand frontier"
[1] 7675
[1] "Elapsed time:"
Time difference of 4.013038 mins
[1] 9
[1] "Expand frontier"
[1] 7683
[1] "Elapsed time:"
Time difference of 4.022829 mins
[1] 8
[1] "Expand frontier"
[1] 7690
[1] "Elapsed time:"
Time difference of 4.045704 mins
[1] 9
[1] "Expand frontier"
[1] 7698
[1] "Elapsed time:"
Time difference of 4.074867 mins
[1] 14
[1] "Expand frontier"
[1] 7711
[1] "Elapsed time:"
Time difference of 4.107039 mins
[1] 12
[1] "Expand frontier"
[1] 7722
[1] "Elapsed time:"
Time difference of 4.13593 mins
[1] 8
[1] "Expand frontier"
[1] 7729
[1] "Elapsed time:"
Time difference of 4.168442 mins
[1] 8
[1] "Expand frontier"
[1] 7736
[1] "Elapsed time:"
Time difference of 4.205933 mins
[1] 12
[1] "Expand frontier"
[1] 7747
[1] "Elapsed time:"
Time difference of 4.235901 mins
[1] 9
[1] "Expand frontier"
[1] 7755
[1] "Elapsed time:"
Time difference of 4.289034 mins
[1] 2
[1]

[1] 10
[1] "Expand frontier"
[1] 8311
[1] "Elapsed time:"
Time difference of 6.557312 mins
[1] 6
[1] "Expand frontier"
[1] 8316
[1] "Elapsed time:"
Time difference of 6.587269 mins
[1] 4
[1] "Expand frontier"
[1] 8319
[1] "Elapsed time:"
Time difference of 6.636757 mins
[1] 8
[1] "Expand frontier"
[1] 8326
[1] "Elapsed time:"
Time difference of 6.668902 mins
[1] 8
[1] "Expand frontier"
[1] 8333
[1] "Elapsed time:"
Time difference of 6.705058 mins
[1] 7
[1] "Expand frontier"
[1] 8339
[1] "Elapsed time:"
Time difference of 6.706775 mins
[1] 3
[1] "Expand frontier"
[1] 8341
[1] "Elapsed time:"
Time difference of 6.744499 mins
[1] 2
[1] "Expand frontier"
[1] 8342
[1] "Elapsed time:"
Time difference of 6.772818 mins
[1] 4
[1] "Expand frontier"
[1] 8345
[1] "Elapsed time:"
Time difference of 6.801161 mins
[1] 5
[1] "Expand frontier"
[1] 8349
[1] "Elapsed time:"
Time difference of 6.829263 mins
[1] 8
[1] "Expand frontier"
[1] 8356
[1] "Elapsed time:"
Time difference of 6.857509 mins
[1] 11
[1

[1] 3
[1] "Expand frontier"
[1] 8685
[1] "Elapsed time:"
Time difference of 9.049711 mins
[1] 6
[1] "Expand frontier"
[1] 8690
[1] "Elapsed time:"
Time difference of 9.051994 mins
[1] 4
[1] "Expand frontier"
[1] 8693
[1] "Elapsed time:"
Time difference of 9.054281 mins
[1] 3
[1] "Expand frontier"
[1] 8695
[1] "Elapsed time:"
Time difference of 9.089301 mins
[1] 4
[1] "Expand frontier"
[1] 8698
[1] "Elapsed time:"
Time difference of 9.138994 mins
[1] 2
[1] "Expand frontier"
[1] 8699
[1] "Elapsed time:"
Time difference of 9.170791 mins
[1] 2
[1] "Expand frontier"
[1] 8700
[1] "Elapsed time:"
Time difference of 9.199249 mins
[1] 2
[1] "Expand frontier"
[1] 8701
[1] "Elapsed time:"
Time difference of 9.231036 mins
[1] 5
[1] "Expand frontier"
[1] 8705
[1] "Elapsed time:"
Time difference of 9.267147 mins
[1] 3
[1] "Expand frontier"
[1] 8707
[1] "Elapsed time:"
Time difference of 9.297005 mins
[1] 2
[1] "Expand frontier"
[1] 8708
[1] "Elapsed time:"
Time difference of 9.327143 mins
[1] 2
[1] 

[1] 3
[1] "Expand frontier"
[1] 8872
[1] "Elapsed time:"
Time difference of 11.26783 mins
[1] 2
[1] "Expand frontier"
[1] 8873
[1] "Elapsed time:"
Time difference of 11.29792 mins
[1] 5
[1] "Expand frontier"
[1] 8877
[1] "Elapsed time:"
Time difference of 11.33485 mins
[1] 5
[1] "Expand frontier"
[1] 8881
[1] "Elapsed time:"
Time difference of 11.33652 mins
[1] 2
[1] "Expand frontier"
[1] 8882
[1] "Elapsed time:"
Time difference of 11.39531 mins
[1] 4
[1] "Expand frontier"
[1] 8885
[1] "Elapsed time:"
Time difference of 11.42496 mins
[1] 3
[1] "Expand frontier"
[1] 8887
[1] "Elapsed time:"
Time difference of 11.45453 mins
[1] 7
[1] "Expand frontier"
[1] 8893
[1] "Elapsed time:"
Time difference of 11.49242 mins
[1] 1
[1] "Expand frontier"
[1] 8893
[1] "Elapsed time:"
Time difference of 11.54563 mins
[1] 3
[1] "Expand frontier"
[1] 8895
[1] "Elapsed time:"
Time difference of 11.57754 mins
[1] 3
[1] "Expand frontier"
[1] 8897
[1] "Elapsed time:"
Time difference of 11.58656 mins
[1] 3
[1] 

...

[1] 0
[1] "Expand frontier"
[1] 280
[1] "Elapsed time:"
Time difference of 2.790596 hours
[1] 0
[1] "Expand frontier"
[1] 279
[1] "Elapsed time:"
Time difference of 2.790624 hours
[1] 0
[1] "Expand frontier"
[1] 278
[1] "Elapsed time:"
Time difference of 2.79065 hours
[1] 0
[1] "Expand frontier"
[1] 277
[1] "Elapsed time:"
Time difference of 2.790676 hours
[1] 0
[1] "Expand frontier"
[1] 276
[1] "Elapsed time:"
Time difference of 2.790701 hours
[1] 0
[1] "Expand frontier"
[1] 275
[1] "Elapsed time:"
Time difference of 2.790725 hours
[1] 0
[1] "Expand frontier"
[1] 274
[1] "Elapsed time:"
Time difference of 2.79075 hours
[1] 0
[1] "Expand frontier"
[1] 273
[1] "Elapsed time:"
Time difference of 2.790871 hours
[1] 0
[1] "Expand frontier"
[1] 272
[1] "Elapsed time:"
Time difference of 2.790896 hours
[1] 0
[1] "Expand frontier"
[1] 271
[1] "Elapsed time:"
Time difference of 2.790921 hours
[1] 0
[1] "Expand frontier"
[1] 270
[1] "Elapsed time:"
Time difference of 2.790944 hours
[1] 0
[1] "E

[1] 0
[1] "Expand frontier"
[1] 188
[1] "Elapsed time:"
Time difference of 2.811807 hours
[1] 0
[1] "Expand frontier"
[1] 187
[1] "Elapsed time:"
Time difference of 2.811841 hours
[1] 0
[1] "Expand frontier"
[1] 186
[1] "Elapsed time:"
Time difference of 2.811867 hours
[1] 0
[1] "Expand frontier"
[1] 185
[1] "Elapsed time:"
Time difference of 2.81202 hours
[1] 0
[1] "Expand frontier"
[1] 184
[1] "Elapsed time:"
Time difference of 2.812047 hours
[1] 0
[1] "Expand frontier"
[1] 183
[1] "Elapsed time:"
Time difference of 2.812072 hours
[1] 0
[1] "Expand frontier"
[1] 182
[1] "Elapsed time:"
Time difference of 2.812096 hours
[1] 0
[1] "Expand frontier"
[1] 181
[1] "Elapsed time:"
Time difference of 2.812209 hours
[1] 0
[1] "Expand frontier"
[1] 180
[1] "Elapsed time:"
Time difference of 2.812234 hours
[1] 0
[1] "Expand frontier"
[1] 179
[1] "Elapsed time:"
Time difference of 2.812257 hours
[1] 0
[1] "Expand frontier"
[1] 178
[1] "Elapsed time:"
Time difference of 2.812281 hours
[1] 0
[1] "

[1] 0
[1] "Expand frontier"
[1] 96
[1] "Elapsed time:"
Time difference of 2.828178 hours
[1] 0
[1] "Expand frontier"
[1] 95
[1] "Elapsed time:"
Time difference of 2.829004 hours
[1] 0
[1] "Expand frontier"
[1] 94
[1] "Elapsed time:"
Time difference of 2.829657 hours
[1] 0
[1] "Expand frontier"
[1] 93
[1] "Elapsed time:"
Time difference of 2.830148 hours
[1] 0
[1] "Expand frontier"
[1] 92
[1] "Elapsed time:"
Time difference of 2.830769 hours
[1] 0
[1] "Expand frontier"
[1] 91
[1] "Elapsed time:"
Time difference of 2.830807 hours
[1] 0
[1] "Expand frontier"
[1] 90
[1] "Elapsed time:"
Time difference of 2.830925 hours
[1] 0
[1] "Expand frontier"
[1] 89
[1] "Elapsed time:"
Time difference of 2.830952 hours
[1] 0
[1] "Expand frontier"
[1] 88
[1] "Elapsed time:"
Time difference of 2.830977 hours
[1] 0
[1] "Expand frontier"
[1] 87
[1] "Elapsed time:"
Time difference of 2.831086 hours
[1] 0
[1] "Expand frontier"
[1] 86
[1] "Elapsed time:"
Time difference of 2.83111 hours
[1] 0
[1] "Expand fron

[1] 0
[1] "Expand frontier"
[1] 3
[1] "Elapsed time:"
Time difference of 2.845343 hours
[1] 0
[1] "Expand frontier"
[1] 2
[1] "Elapsed time:"
Time difference of 2.845366 hours
[1] 0
[1] "Expand frontier"
[1] 1
[1] "Elapsed time:"
Time difference of 2.84539 hours
[1] 0


iteration,elapsedMinutes,frontierSize,elapsedSeconds
<dbl>,<dbl>,<dbl>,<dbl>
0,0,1,NA
1,NA,4342,13.68122
2,NA,4617,16.77101
3,NA,4799,21.38686
4,NA,4946,23.92416
5,NA,5109,27.60626
6,NA,5264,32.22835
7,NA,5374,34.81745
8,NA,5445,39.17074


In [109]:
expansionHistory <- tibble(iteration=0, elapsedMinutes=0, frontierSize=1)
expansionHistory

iteration,elapsedMinutes,frontierSize
<dbl>,<dbl>,<dbl>
0,0,1


In [ ]:
nodeCount <- 1
rootNode <- makeRootNode()
frontier <-tibble(absPath='.', nOccurrences=rootNode$nOccurrences)


node <- rootNode
nextWordIndices <- getOccurrences(node) + 1
nextWords <- words[nextWordIndices,]
parentPath <- node$absPath
parentSequence <- node$sequence
wordCounts <- nextWords %>%
    count(word) %>%
    rename(nOccurrences=n) %>%
    arrange(desc(nOccurrences))

#create dataframe representing new nodes
newNodes <- as.Node(
    wordCounts %>%
        mutate(name=as.character(row_number() + nodeCount)) %>% 
        mutate(absPath = paste(parentPath, name, sep='/'))  %>%
        mutate(pathString = paste0('./', name)) %>%
        mutate(sequence = paste(parentSequence, word, sep=' '))
)

for (child in newNodes$children){ 
    rootNode$AddChildNode(child)
}


In [ ]:
frontier <- newNodes$children[c(100, 3, 1000)]
# frontier[order(sapply(frontier, function(n) n$nOccurrences))]
sapply(frontier, function(n) n$nOccurrences)
frontier[sapply(frontier, function(n) n$nOccurrences > 20)]
frontier

In [ ]:
nodeToExpand <- frontier[[1]]
nodeToExpand$nOccurrences
frontier[-1]

In [ ]:
sequences <- ToDataFrameTable(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')
sequences

In [ ]:
Navigate(rootNode, './4')$absPath
# print(rootNode)

In [ ]:
child <- rootNode$children[[1]]
child$pathString
Navigate(rootNode, './23')
## Why cant navigate??

In [ ]:
nodeCount <- 1
rootNode <- makeRootNode()
# frontier <-tibble(absPath='.', nOccurrences=rootNode$nOccurrences)
# frontier = list(rootNode)
frontier <- tibble(node = c(rootNode), nOccurrences=rootNode$nOccurrences)
# frontier <- tibble(node = newNodes, nOccurrences = sapply(newNodes, function(n) n$nOccurrences))


nodeToExpand <- frontier$node[[1]]

# remove it from the frontier
frontier <- frontier[-1,]

# expand it
newNodes <- expandNode(nodeToExpand)
#     print("new nodes")
#     print(newNodes)

# add new nodes to frontier
#     frontier <- bind_rows(frontier, newNodes)
#dont add nodes below a certain occurrence threshold to the frontier
# newNodes <- newNodes[sapply(newNodes, function(n) n$nOccurrences > 0)]


frontier <- bind_rows(frontier, tibble(node = newNodes, nOccurrences = sapply(newNodes, function(n) n$nOccurrences)))
frontier <- arrange(frontier, desc(nOccurrences))
                                                                              
#keep frontier sorted
# frontier <- frontier[order(sapply(frontier, function(n) n$nOccurrences))]

ToDataFrameTable(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')
       
frontier

In [115]:
nodeToExpand <- frontier$node[[1]]

# remove it from the frontier
frontier <- frontier[-1,]

# expand it
newNodes <- expandNode(nodeToExpand)
#     print("new nodes")
#     print(newNodes)

# add new nodes to frontier
#     frontier <- bind_rows(frontier, newNodes)
#dont add nodes below a certain occurrence threshold to the frontier
# newNodes <- newNodes[sapply(newNodes, function(n) n$nOccurrences > 0)]


frontier <- bind_rows(frontier, tibble(node = newNodes, nOccurrences = sapply(newNodes, function(n) n$nOccurrences)))
frontier <- arrange(frontier, desc(nOccurrences))
                                                                              
ToDataFrameTable(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')
       
frontier

# A tibble: 5 x 2
  word  nOccurrences
  <chr>        <int>
1 thing         1687
2 time          1438
3 eol           1185
4 here          1143
5 way            687


word,nOccurrences,absPath,sequence,level
<chr>,<int>,<chr>,<chr>,<dbl>
eol,2360155,./2,eol,2
eol,1865,./3/2/2,the best eol,4
of,770,./3/2/3,the best of,4
way,646,./3/2/4,the best way,4
thing,632,./3/2/5,the best thing,4
same,12760,./3/3,the same,3
world,10826,./3/4,the world,3
new,10172,./3/5,the new,3
first,9741,./3/6,the first,3


node,nOccurrences
<list>,<int>
<environment: 0x5bb67ed8>,18506
<environment: 0xbc872a28>,18483
<environment: 0x513245f0>,18477
<environment: 0x5304f760>,18329
<environment: 0xbc9492b0>,18246
<environment: 0x50f42c28>,18136
<environment: 0x84ac0398>,18133
<environment: 0x6142bca0>,18060
<environment: 0x503148c8>,17969


In [ ]:
# newNodes[sapply(newNodes, function(n) n$nOccurrences > 5)]
# nodeToExpand$word
sapply(newNodes, function(n) n$nOccurrences > 5)

In [ ]:
nodeToExpand <- frontier$node[[1]]

# remove it from the frontier
frontier <- frontier[-1,]

# expand it
newNodes <- expandNode(nodeToExpand)
#     print("new nodes")
#     print(newNodes)

# add new nodes to frontier
#     frontier <- bind_rows(frontier, newNodes)
#dont add nodes below a certain occurrence threshold to the frontier
# newNodes <- newNodes[sapply(newNodes, function(n) n$nOccurrences > 0)]


frontier <- bind_rows(frontier, tibble(node = newNodes, nOccurrences = sapply(newNodes, function(n) n$nOccurrences)))
frontier <- arrange(frontier, desc(nOccurrences))
                                                                              
ToDataFrameTable(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')
       
frontier

In [ ]:
nodeToExpand <- frontier$node[[1]]

# remove it from the frontier
frontier <- frontier[-1,]

# expand it
newNodes <- expandNode(nodeToExpand)
#     print("new nodes")
#     print(newNodes)

# add new nodes to frontier
#     frontier <- bind_rows(frontier, newNodes)
#dont add nodes below a certain occurrence threshold to the frontier
# newNodes <- newNodes[sapply(newNodes, function(n) n$nOccurrences > 0)]


frontier <- bind_rows(frontier, tibble(node = newNodes, nOccurrences = sapply(newNodes, function(n) n$nOccurrences)))
frontier <- arrange(frontier, desc(nOccurrences))
                                                                              
ToDataFrameTable(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')
       
frontier

In [ ]:
ToDataFrameNetwork(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')

In [ ]:
print(rootNode, 'nOccurrences', limit = 20)

In [ ]:
ToDataFram

In [ ]:
rootNode$children[[2]]

In [ ]:
print(rootNode, 'nOccurrences', limit = 20)

In [127]:
print(rootNode$'the', limit = 20)

NULL


In [ ]:
levels <- rootNode$Get('level', 'pathString')

In [6]:
startTime <- Sys.time()

In [126]:
# explore the generated tree
ToDataFrameNetwork(rootNode, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')

from,to,word,nOccurrences,absPath,sequence,level
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>
1,1/2,eol,2360155,./2,eol,2
1,1/3,the,937405,./3,the,2
1,1/4,to,788645,./4,to,2
1,1/5,i,723447,./5,i,2
1,1/6,a,611358,./6,a,2
1,1/7,you,548089,./7,you,2
1,1/8,and,438538,./8,and,2
1,1/9,for,385348,./9,for,2
1,1/10,in,380376,./10,in,2


In [177]:
# try predict a word. One ...
one <- rootNode$children[Get(rootNode$children, function(node) list(word=node$word) == "one")][[1]]
    
ToDataFrameNetwork(one, 'word', 'nOccurrences', 'absPath', 'sequence', 'level')

from,to,word,nOccurrences,absPath,sequence,level
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>
1/50,1/50/2,of,15618,./50/2,one of,2
1/50,1/50/3,day,2864,./50/3,one day,2
1/50,1/50/4,more,2165,./50/4,one more,2
1/50,1/50/5,is,1512,./50/5,one is,2
1/50,1/50/6,thing,1449,./50/6,one thing,2
1/50,1/50/7,direction,1258,./50/7,one direction,2
1/50,1/50/8,in,1234,./50/8,one in,2
1/50,1/50/9,who,1202,./50/9,one who,2
1/50,1/50/10,i,1116,./50/10,one i,2


In [151]:
getWordNode <- function(parentNode, word) {
    matchingNodes <- rootNode$children[Get(rootNode$children, function(node) list(word=node$word) == word)]
    numMatching <- length(matchingNodes)
    if (numMatching > 1){
        stop("Error: found multiple nodes matching given word")
    } else if(numMatching == 1) {
        node <- matchingNodes[[1]]
    } else {
        node <- NULL
    }
    node
} 

In [154]:

ToDataFrameNetwork(
    getWordNode(rootNode, "hi"),
    'word', 'nOccurrences', 'absPath', 'sequence', 'level'
)

from,to,word,nOccurrences,absPath,sequence,level
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>


In [166]:
node <- getWordNode(rootNode, "hi")
ToDataFrameTree(node,
    'word', 'nOccurrences', 'absPath', 'sequence', 'level'
)

levelName,word,nOccurrences,absPath,sequence,level
<chr>,<chr>,<int>,<chr>,<chr>,<dbl>
385,hi,9876,./385,hi,1


In [170]:
nextWordIndices <- getOccurrences(node) + 1
nextWords <- words[nextWordIndices,]
wordCounts <- nextWords %>%
    count(word) %>%
    rename(nOccurrences=n) %>%
    arrange(desc(nOccurrences))
wordCounts

word,nOccurrences
<chr>,<int>
eol,777
to,511
i,428
there,346
i'm,209
how,182
my,167
thanks,158
and,139
